Importing libraries


In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical

Splitting the training and testing data

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


Preprocessing of data. The dimension of the training data is (60000,28,28). The CNN model will require one more dimension so we reshape the matrix to shape (60000,28,28,1).

In [4]:
x_train=x_train.reshape(x_train.shape[0],28,28,1)
x_test=x_test.reshape(x_test.shape[0],28,28,1)
input_data=(28,28,1)
num_classes= 10


In [5]:

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [6]:
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')
x_train /=255
x_test /= 255
print('x_train shape:', x_train.shape)
print( 'train samples', x_train.shape[0])
print( 'test samples', x_test.shape[0])


x_train shape: (60000, 28, 28, 1)
train samples 60000
test samples 10000


In [7]:
batch_size = 128
#num_classes = 10
epochs = 20

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_data))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [8]:


hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
print("The model has successfully trained")

model.save('number recognition')
print("Saving the model as number recognition")



Epoch 1/20
469/469 [==============================] - 80s 169ms/step - loss: 2.2658 - accuracy: 0.1688 - val_loss: 2.2108 - val_accuracy: 0.4113
Epoch 2/20
469/469 [==============================] - 81s 172ms/step - loss: 2.1783 - accuracy: 0.3220 - val_loss: 2.0966 - val_accuracy: 0.6046
Epoch 3/20
469/469 [==============================] - 81s 173ms/step - loss: 2.0574 - accuracy: 0.4469 - val_loss: 1.9379 - val_accuracy: 0.7105
Epoch 4/20
469/469 [==============================] - 79s 168ms/step - loss: 1.8890 - accuracy: 0.5415 - val_loss: 1.7220 - val_accuracy: 0.7602
Epoch 5/20
469/469 [==============================] - 89s 189ms/step - loss: 1.6721 - accuracy: 0.6116 - val_loss: 1.4589 - val_accuracy: 0.7878
Epoch 6/20
469/469 [==============================] - 82s 174ms/step - loss: 1.4392 - accuracy: 0.6525 - val_loss: 1.1984 - val_accuracy: 0.8036
Epoch 7/20
469/469 [==============================] - 81s 173ms/step - loss: 1.2334 - accuracy: 0.6857 - val_loss: 0.9869 - val_ac

INFO:tensorflow:Assets written to: number recognition\assets


INFO:tensorflow:Assets written to: number recognition\assets


Saving the model as number recognition


In [9]:
score=model.evaluate(x_test,y_test)
print('Test score:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 3s 11ms/step - loss: 0.3972 - accuracy: 0.8935
Test score: 0.39717891812324524
Test accuracy: 0.8934999704360962


In [11]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

model = load_model('number recognition')

def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=400, height=400, bg = "beige", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

app = App()
mainloop()

1/1 [==============================] - 0s 74ms/step
